In [20]:
using NBInclude
nbinclude("../old/ScoreMatching_functions.ipynb")

coordinate_descent (generic function with 1 method)

In [77]:
# Simulate with dimensions ex_dims = [M, D, F, N]. b is assumed given and known, ω=1/T=1

ex_dims = [20, 4, 16, 10]
ex_params = create_rand_params(ex_dims...);
ex_params.ω = 0.25
ex_params.b = log(100)*ones(size(ex_params.b));
out = sample_spikes(getall(ex_params)...);

j_true = J(out, getall(ex_params))

#rand_params = deepcopy(ex_params);
#rand_params.ϕ = 2*π*rand(size(rand_params.ϕ))
rand_params = create_rand_params(ex_dims...);

j_rand = J(out, getall(rand_params)) # Score matching cost

@show j_true[1], j_rand[1]

@show size(out[1][1])
@show size(j_true[2][1,1])

par = getall(rand_params);

(N,M,D) = (10,20,4)
(j_true[1],j_rand[1]) = (-16398.72658435881,1.3911402844921388e6)
size((out[1])[1]) = (235,)
size((j_true[2])[1,1]) = (16,235)


In [78]:
# Plot the true rates
# Plot data, true rate function and estimated rate function for a neuron on given trials
using PlotlyJS

num_neur = 2
trials = 1:5
dim_latent = 2

#Plot estimated
to_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
rate_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
rate_rand_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
rate_est_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
latent_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
latent_est_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
colors = ["rgb(0,0,255)", "rgb(0,255,0)", "rgb(255,0,0)", "rgb(128,128,0)", "rgb(0,128,128)"]
for i1 = trials
    # Plot data
    to_plot[i1] = scatter(;x=collect(out[i1][num_neur]), y=i1*collect(ones(size(out[i1][num_neur]))), mode="markers", marker_color=colors[mod(i1,5)+1], yaxis="y2")
    
    # Compute underlying rate and latent functions
    rate = lambda_functions(0:0.01:ex_params.T, ex_params.C, ex_params.θ; b=ex_params.b, phi=ex_params.ϕ[:,i1], omega=ex_params.ω)
  
    # Plot underlying functions
    rate_plot[i1] = scatter(; 
        x=collect(0:0.01:ex_params.T), 
    y=collect(rate[num_neur,:]),
    line_color = colors[mod(i1,5)+1],
    yaxis = "y1"
)

end

lo = Layout(;xaxis_range=[0,1])

#plt = plot([Plot(to_plot[trials]); Plot(rate_plot[trials]);  Plot(rate_est_plot[trials]);  Plot(latent_plot[trials]);  Plot(latent_est_plot[trials])])
plt = plot([Plot(to_plot[trials], lo); Plot(rate_plot[trials], lo)])

relayout!(plt, height=700)

plt

# Now fit this with the new model

In [79]:
nbinclude("../src/PoissonProcessEstimation.ipynb")
import PoissonProcessEstimation

In [80]:
KernList = [MLKernels.GaussianKernel(400.0), MLKernels.GaussianKernel(200.0), MLKernels.GaussianKernel(100.0), MLKernels.GaussianKernel(50.0)]

4-element Array{MLKernels.ExponentialKernel{Float64,:γ1},1}:
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=400.0,γ=1.0)
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=200.0,γ=1.0)
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=100.0,γ=1.0)
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=50.0,γ=1.0) 

In [81]:
θ = PoissonProcessEstimation.create_rand_params(ex_dims[4],ex_dims[1],length(KernList),[20,10,8,8]);

In [82]:
θ_orig = θ
θ_opt = deepcopy(θ)
data = out
#C, α, u, N, M, D, J = PoissonProcessEstimation.name_params(θ_opt);

In [83]:
# Run the optimisation
@time Ktu, dtKtu, ddtKtu, Kuu, I = 
    PoissonProcessEstimation.optimise!(data, θ_opt, KernList, η_RKHS=1e1, η_Cm=1e1, num_iters = 25, verbose=false);

  7.597193 seconds (198.81 M allocations: 3.106 GB, 5.59% gc time)
Kernels have been built

Starting optimisation, initial log-likelihood is --

  3.675884 seconds (2.23 M allocations: 3.507 GB, 18.51% gc time)
α and C are set, log-likelihood is -1.5219416771348672e11

  0.000125 seconds (706 allocations: 14.797 KB)
Optimisation done, final log-likelihood is --

 14.641958 seconds (243.22 M allocations: 7.357 GB, 8.42% gc time)


In [84]:
# Plot the data (top), estimated rates (middle), 
PoissonProcessEstimation.plot_params(data, θ_opt, θ_orig, KernList, num_neur=2, trials=1:5, dim_latent=[])

In [74]:
# Plot the true rates
# Plot data, true rate function and estimated rate function for a neuron on given trials
using PlotlyJS

num_neur = 2
trials = 1:5
dim_latent = 2

#Plot estimated
to_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
rate_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
rate_rand_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
rate_est_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
latent_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
latent_est_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(out,1))
colors = ["rgb(0,0,255)", "rgb(0,255,0)", "rgb(255,0,0)", "rgb(128,128,0)", "rgb(0,128,128)"]
for i1 = trials
    # Plot data
    to_plot[i1] = scatter(;x=collect(out[i1][num_neur]), y=i1*collect(ones(size(out[i1][num_neur]))), mode="markers", marker_color=colors[mod(i1,5)+1], yaxis="y2")
    
    # Compute underlying rate and latent functions
    rate = lambda_functions(0:0.01:ex_params.T, ex_params.C, ex_params.θ; b=ex_params.b, phi=ex_params.ϕ[:,i1], omega=ex_params.ω)
  
    # Plot underlying functions
    rate_plot[i1] = scatter(; 
        x=collect(0:0.01:ex_params.T), 
    y=collect(rate[num_neur,:]),
    line_color = colors[mod(i1,5)+1],
    yaxis = "y1"
)

end

lo = Layout(;xaxis_range=[0,1])

#plt = plot([Plot(to_plot[trials]); Plot(rate_plot[trials]);  Plot(rate_est_plot[trials]);  Plot(latent_plot[trials]);  Plot(latent_est_plot[trials])])
plt = plot([Plot(to_plot[trials], lo); Plot(rate_plot[trials], lo)])

relayout!(plt, height=700)

plt

In [75]:
[PoissonProcessEstimation.subspace(θ_opt.C[:,:], ex_params.C)[1] PoissonProcessEstimation.subspace(randn(size(θ_opt.C)), ex_params.C)[1]]

4x2 Array{Float64,2}:
  6.83101  42.0642
 11.2647   56.4547
 16.1417   65.5294
 49.3717   87.6561

In [76]:
# Check for correlation of predicted firing rates
N = ex_dims[4]
M = ex_dims[1]
D = ex_dims[2]

opt_corrs = zeros(N,M)
rand_corrs = zeros(N,M)
rate_true = Array(Any,(N,M))

for n = 1:N
    rate_true_n= lambda_functions(0:0.01:1, ex_params.C, ex_params.θ; b=ex_params.b, phi=ex_params.ϕ[:,n], omega=ex_params.ω)
    for m = 1:M
        rate_true[n,m] = (rate_true_n[m,:])[:]
    end
end
rate_est = PoissonProcessEstimation.firing_rates(0:0.01:1, θ_opt, KernList, n_range=1:N, m_range=1:M, d_range=1:D)
rate_rand = PoissonProcessEstimation.firing_rates(0:0.01:1, θ_orig, KernList, n_range=1:N, m_range=1:M, d_range=1:D)
for n = 1:N
    for m = 1:M
        opt_corrs[n,m] = cor(rate_true[n,m], rate_est[n,m])
        rand_corrs[n,m] = cor(rate_true[n,m], rate_rand[n,m])
    end
end

# Show the computed average correlation per neuron
[mean(opt_corrs,1)' mean(rand_corrs,1)']

20x2 Array{Float64,2}:
 0.414305  -0.110761  
 0.628798   0.00323549
 0.738947   0.089105  
 0.562515  -0.13955   
 0.665456   0.0574403 
 0.630035   0.0518279 
 0.547908   0.0634929 
 0.759172  -0.00582173
 0.679887   0.04173   
 0.57868    0.0663747 
 0.531264   0.131263  
 0.665503  -0.0125651 
 0.786929  -0.0703831 
 0.787618   0.0107542 
 0.678173  -0.0283011 
 0.675151   0.0123129 
 0.68749    0.0315806 
 0.629831   0.123776  
 0.458662   0.079491  
 0.379376   0.0697831 